In [1]:
# Sample user submission
message = '''
    Create a count for each character in the string.
    For each character:
    Convert it to lowercase.
    Count how many times it appears.
    Set total_length to 0 and odd_count to 0.
    For each character count:
    Add the even part of the count to total_length.
    If the count is odd, add 1 to odd_count.
    If odd_count is greater than 0, add 1 to total_length.
    Return total_length.
'''


In [2]:
# Load JSON data
import json

with open('sample_problem.json', 'r') as file:
    problem_data = json.load(file)

# Define system_prompt
prompt = f'''
    You are a professor who specializes in solving code problems.
    The user will submit in words their approach to solving the problem.
    Analyze the user's submission and provide feedback.

    Output your feedback in JSON format with the following structure:
    {{
        "title": "Create a title of the analysis",
        "summary": "Create a summary of the analysis",
        "analysis": ["Create a detailed analysis on the user's submission in a list of strings"],
        "suggestions": ["Create suggestions for the user to improve their method in a list of strings"],
        "score": "An integer score: 1 (incorrect), 2 (partially correct), 3 (correct)"
    }}

    Here are the problem details:
    Category: 
    {problem_data['category']}

    Subcategory: 
    {problem_data['subcategory']}

    Difficulty:
    {problem_data['difficulty']}

    Title: 
    {problem_data['title']}

    Description: 
    {problem_data['description']}

    Constraints: 
    {chr(10).join([f'{constraint}' for constraint in problem_data['constraints']])}

    Examples:
    {chr(10).join([f'Input: {example['input']}, Output: {example['output']}, Explanation: {example['explanation']}' for example in problem_data['examples']])}

    Here is the user's submission:
    {message}

    Ensure your output only contains the JSON object without any additional text.
    Provide feedback directly addressing the user as ‘you’ instead of referring to them in the third person.
'''


In [3]:
import requests

# For faster but less comprehensive responses use llama3.1:8b
# model = 'llama3.1:8b'
model = 'llama3.1:8b-instruct-q3_K_S'
url = 'http://127.0.0.1:11434/api/generate'

data = {
    'model': model,
    'prompt': prompt
}

# Send a POST request to the specified URL with the data, enabling streaming
response = requests.post(url, json=data, stream=True)

# Iterate over the lines in the streamed response
for line in response.iter_lines():
    if line:  # Check if the line is not empty
        result = json.loads(line)  # Parse the JSON response from the model
        # Output the 'response' field from the result. 
        # 'end'='' ensures that output remains on the same line,
        # 'flush=True' forces the output buffer to be flushed immediately.
        print(result.get('response', ''), end='', flush=True)  
        
        # Check if the 'done' flag is set to True, indicating the model has finished responding
        if result.get('done'):
            break  # Exit the loop if the response is complete


{
    "title": "Longest Palindrome Analysis",
    "summary": "Your approach is partially correct, but there are some issues that need attention.",
    "analysis": [
        "You correctly convert each character to lowercase using the built-in lower() method.",
        "However, you don't need to count how many times each character appears. Instead, directly calculate the maximum length of the palindrome based on the characters' frequency.",
        "Your code sets total_length and odd_count variables, but they are not necessary. You can simplify your logic by only considering the character counts.",
        "When adding the even part of the count to total_length, you should also consider other possible palindromes that can be formed from the current character's count."
    ],
    "suggestions": [
        "Consider using a dictionary to store the frequency of each character.",
        "Rethink your logic for calculating the maximum length of the palindrome based on character frequencies